<a href="https://colab.research.google.com/github/moradza/Explanatory-Masking-for-Deep-Learning/blob/main/ExplanatoryMasksforNeuralNetworkInterpretability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explanatory Deep Learning
Two different methods are studied on MNIST dataset to find important feature in image for deep learning classification. Further details inside following references.

1. Masking [ref](https://arxiv.org/pdf/1911.06876.pdf) 
2. Saliancy Map [ref](https://arxiv.org/abs/1610.02391) 

**future work: CNN and Graph Neural Network**<br> 
*additional documentations in the future*

In [1]:
import numpy as np
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import layers 
import matplotlib.pyplot as plt


model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [2]:
original_dim = 784

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32")  / 255
x_test = x_test.reshape(x_test.shape[0], 784).astype("float32") 


In [3]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [4]:
model.fit(x_train, y_train, epochs=40)

Epoch 1/40
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3798 - accuracy: 0.8896
Epoch 2/40
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0976 - accuracy: 0.9708
Epoch 3/40
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0637 - accuracy: 0.9808
Epoch 4/40
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0420 - accuracy: 0.9872
Epoch 5/40
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0295 - accuracy: 0.9910
Epoch 6/40
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0246 - accuracy: 0.9927
Epoch 7/40
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0171 - accuracy: 0.9948
Epoch 8/40
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0137 - accuracy: 0.9956
Epoch 9/40
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0088 - accuracy: 0.9977
Epoch 10/40
1875/1875 [==============================] - 4s 2ms/step - loss: 0.010

In [5]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 38.1366 - accuracy: 0.9822

Test accuracy: 0.982200026512146


In [6]:
# model.get_config()

In [7]:
layer_output = model.get_layer('dense').output
layer_output.trainable =False
model.trainable = False
explainer_layer1 = Dense(int(original_dim/2), activation='relu')(layer_output)#
mask = Dense(original_dim, activation='sigmoid',activity_regularizer=tf.keras.regularizers.L1L2(l1=0.00001, l2=0.00001))(explainer_layer1)#
new_input = tf.keras.layers.Multiply()([mask, model.input])
class_new = model(new_input, training=False)
model2 = Model(model.input, outputs=class_new)


In [8]:
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model2.fit(x_train, y_train, epochs=30)

Epoch 1/30
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0496 - accuracy: 0.9922
Epoch 2/30
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0208 - accuracy: 0.9987
Epoch 3/30
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0176 - accuracy: 0.9983
Epoch 4/30
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0140 - accuracy: 0.9988
Epoch 5/30
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0109 - accuracy: 0.9989
Epoch 6/30
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0082 - accuracy: 0.9994
Epoch 7/30
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0080 - accuracy: 0.9990
Epoch 8/30
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0087 - accuracy: 0.9989
Epoch 9/30
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0081 - accuracy: 0.9988
Epoch 10/30
1875/1875 [==============================] - 12s 7ms/step - l

In [ ]:
test_loss, test_acc = model2.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
# model2.get_config()

In [ ]:
layer_output = model2.get_layer('multiply').output
modelexplainer = Model(model2.input, outputs=layer_output)

In [ ]:
model.summary(), model2.summary()

In [ ]:
masked_img = modelexplainer.predict(x_train)
img = x_train
def plot_image(img):
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(1-img.reshape((28,28)), cmap='gray')
    
num_rows = 8
num_cols = 4
num_images = num_rows*num_cols
plt.figure(figsize=(7*num_cols, 7*num_rows))
for i in range(num_images):
    plt.subplot(2*num_rows, 2*num_cols, 2*i+1)
    plot_image(img[i])
    plt.subplot(2*num_rows, 2*num_cols, 2*i+2)
    plot_image(masked_img[i])
plt.tight_layout()
plt.show()

In [ ]:
def saliency(inputs):
    inps = tf.Variable(inputs)
    with tf.GradientTape() as tape:
        prediction = model(inps)
    return tape.gradient(prediction,inps)

In [ ]:
saliency(x_train[:3]).shape

In [ ]:
saliency_img = saliency(x_train).numpy()
img = x_train

num_rows = 8
num_cols = 4
num_images = num_rows*num_cols
plt.figure(figsize=(5*num_cols, 5*num_rows))
for i in range(num_images):
    plt.subplot(2*num_rows, 2*num_cols, 2*i+1)
    plot_image(img[i])
    plt.subplot(2*num_rows, 2*num_cols, 2*i+2)
    plot_image(saliency_img[i])#/np.max(saliency_img[i]))
plt.tight_layout()
plt.show()

In [ ]:
num_rows = 10
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(7*num_cols, 7*num_rows))
for i in range(num_images):
    plt.subplot(3*num_rows, 3*num_cols, 3*i+1)
    plot_image(saliency_img[i])
    plt.subplot(3*num_rows, 3*num_cols, 3*i+2)
    plot_image(img[i])
    plt.subplot(3*num_rows, 3*num_cols, 3*i+3)
    plot_image(masked_img[i])
plt.tight_layout()
plt.show()

In [ ]:
train_loss, train_acc = model2.evaluate(masked_img,  y_train, verbose=2)
print("Accuracy of masked images: %.4f", train_acc)